# Trend Analysis on Twitter live data using Spark Streaming

Import sparkContext & StreamingContext from PySpark library.

In [1]:
import findspark
findspark.init()

In [2]:
from __future__ import print_function
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

Create a sparkContext with AppName "SteramingTwitterAnalysis".<br>
Setting the LogLevel of SparkContext to ERROR. This will not print all the logs which are INFO or WARN level.<br>
Create Spark Streaming Context using SC (spark context). parameter 10 is the batch interval. <br>
Every 10 second the analysis will be done.

In [3]:
sc = SparkContext(appName="SteramingTwitterAnalysis")
sc.setLogLevel("ERROR")
ssc = StreamingContext(sc, 10)

Connect to socket broker using ssc (spark streaming context)<br>
Host : "172.31.20.58" (localhost) & port : 5555

In [4]:
socket_stream = ssc.socketTextStream("192.168.56.1", 5555)

window function parameter sets the Window length. All the analysis will be done on tweets stored for 60 secs.

In [5]:
lines = socket_stream.window( 20 )

### Process the Stream:
1. Receives tweet message, stored in lines. **Input DStream**
2. splits the messages into words. **Apply transformation on DStream : flatMap**
3. filters all the words which start with a hashtag(#). **transformation : filter**
4. converts the words to lowercase. **transformation : map**
5. maps each tag to (word, 1). **transformation : map**
6. then reduces and counts occurrences of each hash tag. (action : reduceByKey) hashtags = **output DStream**

In [6]:
hashtags = lines.flatMap( lambda text: text.split( " " ) ).filter( lambda word: word.lower().startswith("#") ).map( lambda word: ( word.lower()
, 1 ) ).reduceByKey( lambda a,b:a+b)

Sort the hashtags based on the counts in decreasing order

In [7]:
author_counts_sorted_dstream = hashtags.transform(lambda foo:foo.sortBy(lambda x:x[0].lower()).sortBy(lambda x:x[1],ascending=False))

Print the final analysis: Most popular hashtags on streaming twitter data

In [8]:
author_counts_sorted_dstream.pprint()

### Starting the Spark Streaming:
Spark Streaming code we have written till now will not execute, untill we start the ssc.<br>
ssc.start() will start the spark streaming context. This is the Action for the whole code. <br>
Now it'll create the lineage & DAG & do the lazy evaluation & start running the whole sequesnce of code.


In [9]:
ssc.start()

awaitTermination() is very important to stop the SSC.<br> 
When we kill this python process then this signal will be sent to awaitTermination() function.<br> 
it will finally stop the spark streaming job.

In [10]:
ssc.awaitTermination()

-------------------------------------------
Time: 2020-10-03 15:19:10
-------------------------------------------
('#covid19', 2)
('#accountabilitymatters:', 1)
('#coronavirus', 1)
('#presidenttrump…', 1)
('#trump', 1)
('#trump2020', 1)
('#trump2020landslidevictory', 1)
('#trumpcovid19', 1)
('#weloveyou', 1)

-------------------------------------------
Time: 2020-10-03 15:19:20
-------------------------------------------
('#covid19', 6)
('#trump', 6)
('##', 2)
('#accountabilitymatters:', 2)
('#covid-19', 2)
('#trumphascovid', 2)
('#95minutesintrumpsamerica', 1)
('#boyfriendday', 1)
('#bunkerboytrumpdespite', 1)
('#checry', 1)
...

-------------------------------------------
Time: 2020-10-03 15:19:30
-------------------------------------------
('#trump', 35)
('#covid19', 17)
('#covid-19', 6)
('#get2getherexperience', 5)
('#hitrefresh', 5)
('#humantraffickingawareness', 5)
('#coronavirus', 4)
('#trumpcovid', 4)
('#covid', 3)
('#prayfortrump', 3)
...

-------------------------------------

Py4JJavaError: An error occurred while calling o23.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "C:\Users\Admin\Spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\streaming\util.py", line 68, in call
    r = self.func(t, *rdds)
  File "C:\Users\Admin\Spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\streaming\dstream.py", line 173, in takeAndPrint
    taken = rdd.take(num + 1)
  File "C:\Users\Admin\Spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\rdd.py", line 1446, in take
    res = self.context.runJob(self, takeUpToNumLeft, p)
  File "C:\Users\Admin\Spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\context.py", line 1118, in runJob
    sock_info = self._jvm.PythonRDD.runJob(self._jsc.sc(), mappedRDD._jrdd, partitions)
  File "C:\Users\Admin\Spark\spark-3.0.1-bin-hadoop2.7\python\lib\py4j-0.10.9-src.zip\py4j\java_gateway.py", line 1305, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  Fi

-------------------------------------------
Time: 2020-10-03 15:23:10
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:23:20
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:23:30
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:23:40
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:23:50
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:24:00
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:24:10
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:24:20
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:24:30
----------

-------------------------------------------
Time: 2020-10-03 15:35:10
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:35:20
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:35:30
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:35:40
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:35:50
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:36:00
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:36:10
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:36:20
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:36:30
----------

-------------------------------------------
Time: 2020-10-03 15:47:10
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:47:20
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:47:30
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:47:40
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:47:50
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:48:00
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:48:10
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:48:20
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:48:30
----------

-------------------------------------------
Time: 2020-10-03 15:59:10
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:59:20
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:59:30
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:59:40
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 15:59:50
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:00:00
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:00:10
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:00:20
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:00:30
----------

-------------------------------------------
Time: 2020-10-03 16:11:20
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:11:30
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:11:40
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:11:50
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:12:00
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:12:10
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:12:20
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:12:30
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:12:40
----------

-------------------------------------------
Time: 2020-10-03 16:23:20
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:23:30
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:23:40
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:23:50
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:24:00
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:24:10
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:24:20
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:24:30
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:24:40
----------

-------------------------------------------
Time: 2020-10-03 16:35:20
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:35:30
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:35:40
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:35:50
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:36:00
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:36:10
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:36:20
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:36:30
-------------------------------------------

-------------------------------------------
Time: 2020-10-03 16:36:40
----------